In [1]:
import requests
api_url = 'http://127.0.0.1:5000'

In [2]:
r = requests.get(f'{api_url}/models/types')
print(r, r.text)
r = requests.get(f'{api_url}/models/types/Logit')
print(r, r.text)
r = requests.get(f'{api_url}/models/fitted')
print(r, r.text)
r = requests.get(f'{api_url}/models/fitted/Cox')
print(r, r.text)

<Response [200]> [{"name":"Logit","type":"parametric"},{"name":"Kaplan-Meier","type":"nonparametric"},{"name":"Cox","type":"semiparametric"}]

<Response [200]> {"name":"Logit","type":"parametric"}

<Response [200]> {"Logit": ["221211_233855", "221213_230043"], "Kaplan-Meier": ["221213_230046", "221211_233855"], "Cox": ["221213_230047", "221211_233855"]}

<Response [200]> ["221213_230047","221211_233855"]



In [1]:
# Run cells below from parent dir of "tests"
os.chdir(os.path.dirname(os.getcwd()))

In [2]:
from models.survival import Model, MODEL_TYPES

model = Model()
model.data.head(5)

,index,nAgrmntId,dtAgrmntOpen,dtAgrmntClose,nMonth,ctgrCardLevel,ctgrSalesChannel,dmMassOffer,nIsPayroll,dtReport,dmIsExpired
0,0,0,2021-03-31,NaT,1,5,3,0,1,2021-03-31,0
1,1,0,2021-03-31,NaT,2,5,3,0,1,2021-04-30,0
2,2,0,2021-03-31,NaT,3,5,3,0,1,2021-05-31,0
3,3,0,2021-03-31,NaT,4,5,3,0,1,2021-06-30,0
4,4,0,2021-03-31,NaT,5,5,3,0,1,2021-07-31,0


In [3]:
model.fitted

{'Logit': ['221211_233855', '221213_230043'],
 'Kaplan-Meier': ['221213_230046', '221211_233855'],
 'Cox': ['221213_230047', '221211_233855']}

In [4]:
model.restore(model_type='Kaplan-Meier', model_uid='221211_233855')

'Load model from ./.tmp/storage/Kaplan-Meier/221211_233855'

In [5]:
model.model_type

'Kaplan-Meier'

In [3]:
model.set_model_type('Logit')
spec = ['ctgrCardLevel','ctgrSalesChannel','dmMassOffer','nIsPayroll']
model.fit(spec, fit_intercept=True)
print(model.summary())
model.save()

                           Logit Regression Results                           
Dep. Variable:            dmIsExpired   No. Observations:                20731
Model:                          Logit   Df Residuals:                    20726
Method:                           MLE   Df Model:                            4
Date:                Tue, 13 Dec 2022   Pseudo R-squ.:                0.007994
Time:                        23:00:43   Log-Likelihood:                -1020.6
converged:                       True   LL-Null:                       -1028.8
Covariance Type:            nonrobust   LLR p-value:                  0.002472
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
ctgrCardLevel        0.3079      0.076      4.044      0.000       0.159       0.457
ctgrSalesChannel    -0.0960      0.071     -1.360      0.174      -0.234       0.042
dmMassOffer          0.0361 

/Users/user/projects/MLOps/mlops_restapi/models/survival.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['intercept'] = np.ones(X.shape[0])


'Saved model to ./.tmp/Logit/221213_230043'

In [4]:
model.set_model_type('Kaplan-Meier')
model.fit()
print(model.summary())
model.save()

          KM_estimate
timeline             
0.0          1.000000
1.0          1.000000
2.0          1.000000
3.0          1.000000
4.0          1.000000
...               ...
74.0         0.894666
75.0         0.894666
76.0         0.894666
77.0         0.894666
78.0         0.894666

[79 rows x 1 columns]


'Saved model to ./.tmp/Kaplan-Meier/221213_230046'

In [5]:
model.set_model_type('Cox')
spec = ['ctgrCardLevel','ctgrSalesChannel','dmMassOffer','nIsPayroll']
model.fit(specification=spec, id_col='nAgrmntId', stop_col='nMonth')
print(model.summary())
model.save()

<lifelines.CoxModelFitter: fitted with 20731 periods, 500 subjects, 179 events>
         event col = 'dmIsExpired'
number of subjects = 500
 number of periods = 20731
  number of events = 179
partial log-likelihood = -945.76
  time fit was run = 2022-12-13 20:00:47 UTC

---
                   coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                
ctgrCardLevel      0.60       1.81       0.09             0.43             0.76                 1.53                 2.14
ctgrSalesChannel  -0.11       0.90       0.07            -0.25             0.03                 0.78                 1.03
dmMassOffer        0.87       2.38       0.40             0.08             1.65                 1.08                 5.22
nIsPayroll         0.25       1.28       0.15            -0.05             0.55                 0.95                 1.74

                   cmp to     z      p   -log2(p)
covariate                                        
ctgrCardLevel        0.00  6.99 <0.005      38.41
ctgrSalesChannel     0.00 -1.50   0.13       2.90
dmMassOffer          0.00  2.16   0.03       5.02
nIsPayroll           0.00  1.63   0.10       3.27
---
Partial AIC = 1899.51
log-likelihood ratio test = 45.57 on 4 df
-log2(p) of ll-ratio test = 28.30

None


'Saved model to ./.tmp/Cox/221213_230047'